In [32]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import sys

dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')

from interpolazione import Interpolazione, final_val

dir_path = os.path.abspath('')
FILE = dir_path + '/data/' + 'R_RLC.csv'
fr = pd.read_csv(FILE)

In [33]:
Vin = np.array(fr['Vin']) # V
VR = np.array(fr['VR']) # V
VLC = np.array(fr['V_LC']) # V
ν = np.array(fr['Freq (Hz)']) # Hz 
ω = 2*np.pi*ν # rad/s
ΦR = np.array(fr['faseR (deg)'], dtype=np.float64) * (np.pi / 180) # rad
ΦLC = np.array(fr['faseLC (deg)'], dtype=np.float64) * (np.pi / 180) # rad


sigma_Vin = np.array(fr['sigmaV_in']) # media 4 = sqrt(16)
sigma_VR = np.array(fr['sigmaV_R'])  # media 4 = sqrt(16)
sigma_VLC = np.array(fr['sigmaV_LC']) 

R = 9950 # ohm
sigmaR = R * 0.01 # errore associato ohmmetro
RL = 40
sigmaRL = RL * 0.01

sigma_ΦR = np.array(fr['sigmaF_R'], dtype=np.float64) * (np.pi/180.0) # rad
sigma_ΦLC = np.array(fr['sigmaF_LC'], dtype=np.float64) * (np.pi/180.0) # rad

###### Calcolo funzioni di trasferimento ######

modHR = VR/Vin
sigma_modHR = np.sqrt(((VR/(Vin**2))*sigma_Vin)**2 + ((sigma_VR/Vin)**2))

modHCL = VLC/Vin
sigma_modHCL = np.sqrt(((VLC/(Vin**2))*sigma_Vin)**2 + ((sigma_VLC/Vin)**2))
# print(modHR,sigma_modHR)

##### SISTEMAZIONE PHIR #######

# quando PHI_R è negativo, aggiungo 2pi

shift = lambda x: x + np.pi if x < 0 else x
shift = np.vectorize(shift)
ΦR = shift(ΦR)

- $|H_R(\omega)| = \frac{\omega CR}{\sqrt{(1-\omega^2LC)^2 + \omega^2C^2 (R+R_L)^2}}$

- $\angle H_R(\omega) = \frac{\pi}{2}-arctan(\frac{\omega C(R+R_L)}{1-\omega^2LC})$

In [34]:
def mod_HR_f(omega, C,L):
    numerator = omega * C * R
    denominator = np.sqrt((1 - (omega**2) * L * C)**2 + (omega**2) * (C**2) * (R + RL)**2)
    result = numerator / denominator
    return result

def arg_HR_f(omega, C,L):
    numerator = omega * C * (R + RL)
    denominator = 1 - (omega**2) * L * C
    angle = np.pi/2-np.arctan(numerator / denominator)
    return angle

In [35]:
fit_mod_HR = Interpolazione(ω,modHR,mod_HR_f,sigma_modHR,p0=[10*10**-9,0.04])
fit_arg_HR = Interpolazione(ω,ΦR,arg_HR_f,sigma_ΦR,p0=[10*10**-9,0.04])
# print(fit_mod_HR)
# print(fit_arg_HR)

print("---------------------------MODULO R -------------------------")
print('chi2: ',fit_mod_HR.rchisquare)
print("---------------------------FASE R -------------------------")
print('chi2: ',fit_arg_HR.rchisquare)
print('------------------------- VALORI RICAVATI DAL FIT MODULO HR -----------------------------')
print('C = ',final_val(fit_mod_HR.bval[0],fit_mod_HR.sigma_bval[0],decimals=3,exp=-10,udm='F'))
print('L = ',final_val(fit_mod_HR.bval[1],fit_mod_HR.sigma_bval[1],decimals=3,exp=-3,udm='Hr'))

print('------------------------- VALORI RICAVATI DAL ARG HR -----------------------------')
print('C = ',final_val(fit_arg_HR.bval[0],fit_arg_HR.sigma_bval[0],decimals=3,exp=-10,udm='F'))
print('L = ',final_val(fit_arg_HR.bval[1],fit_arg_HR.sigma_bval[1],decimals=3,exp=-3,udm='Hr'))

---------------------------MODULO R -------------------------
chi2:  0.51
---------------------------FASE R -------------------------
chi2:  1.2
------------------------- VALORI RICAVATI DAL FIT MODULO HR -----------------------------
C =  (97.0 ± 2.936)e-10 F
L =  (48.723 ± 1.496)e-3 Hr
------------------------- VALORI RICAVATI DAL ARG HR -----------------------------
C =  (94.678 ± 5.204)e-10 F
L =  (53.444 ± 2.956)e-3 Hr


In [36]:
# x = np.logspace(2.5, 6.3, 1000)
# y = mod_HR_f(x, *fit_mod_HR.bval)

# plt.plot(x, y, label='Fit',color='yellowgreen')
# plt.errorbar(ω, modHR, yerr=fit_mod_HR.sigmaY, fmt='.',color='darkolivegreen')
# plt.xlabel('$\omega$')
# plt.ylabel('$|H_R(\omega)|$')
# plt.xscale('log')
# plt.title('$|H_R(\omega)|$ \t $\\tilde{\chi}_0^2 = $' + str(fit_mod_HR.rchisquare))
# plt.show()

In [37]:
# x = np.logspace(2.5, 6.3, 1000)
# y = arg_HR_f(x,10*10**-9,0.04)
# y = arg_HR_f(x, *fit_arg_HR.bval)

# plt.plot(x, y, label='Fit',color='yellowgreen')
# plt.errorbar(ω, ΦR, yerr=fit_arg_HR.sigmaY, fmt='.',color='darkolivegreen')
# plt.xlabel('$\omega$')
# plt.ylabel('$|H_R(\omega)|$')
# plt.xscale('log')
# plt.title('$|H_R(\omega)|$ \t $\\tilde{\chi}_0^2 = $' + str(fit_arg_HR.rchisquare))
# plt.show()

### $H_{CL}(\omega) = $

- $|H_{CL}(\omega)| = \frac{\sqrt{(1-\omega^2 LC) + \omega^2C^2 R_L^2}}{\sqrt{(1-\omega^2LC)^2 + \omega^2C^2 (R+R_L)^2}}$
- $\angle H_{CL}(\omega) = arctan(\frac{\omega CR_L}{1-\omega^2LC}) - arctan(\frac{\omega C(R+R_L)}{1-\omega^2LC})$

In [38]:
def mod_HCL_f(omega, C, L):
    numerator = np.sqrt((1 - omega**2 * L * C) + omega**2 * C**2 * RL**2)
    denominator = np.sqrt((1 - omega**2 * L * C)**2 + omega**2 * C**2 * (R + RL)**2)
    return numerator / denominator

def arg_HCL_f(omega, C, L):
    term1 = np.arctan(omega * C * RL / (1 - omega**2 * L * C))
    term2 = np.arctan(omega * C * (R + RL) / (1 - omega**2 * L * C))
    return term1 - term2

In [40]:
print(ω)

[6.28318531e+02 8.79645943e+02 1.25663706e+03 1.75929189e+03
 2.51327412e+03 3.51858377e+03 5.02654825e+03 7.09999940e+03
 1.00530965e+04 1.41999988e+04 2.01061930e+04 2.82743339e+04
 4.02123860e+04 5.65486678e+04 8.04247719e+04 1.13097336e+05
 1.60849544e+05 2.27451308e+05 3.21699088e+05 4.54902616e+05
 6.43398175e+05 9.09805232e+05 1.28679635e+06]


In [41]:
fit_mod_HCL = Interpolazione(ω,modHCL,mod_HCL_f,sigma_modHCL,p0=[10*10**-9,0.04])
fit_arg_HCL = Interpolazione(ω,ΦLC,arg_HCL_f,sigma_ΦLC,p0=[10*10**-9,0.04])
# print(fit_mod_HR
# print(fit_arg_HR)

print("---------------------------MODULO R -------------------------")
print('chi2: ',fit_mod_HCL.rchisquare)
print("---------------------------FASE R -------------------------")
print('chi2: ',fit_arg_HCL.rchisquare)
print('------------------------- VALORI RICAVATI DAL FIT MODULO HR -----------------------------')
print('C = ',final_val(fit_mod_HCL.bval[0],fit_mod_HCL.sigma_bval[0],decimals=3,exp=-10,udm='F'))
print('L = ',final_val(fit_mod_HCL.bval[1],fit_mod_HCL.sigma_bval[1],decimals=3,exp=-3,udm='Hr'))

print('------------------------- VALORI RICAVATI DAL ARG HR -----------------------------')
print('C = ',final_val(fit_arg_HCL.bval[0],fit_arg_HCL.sigma_bval[0],decimals=3,exp=-10,udm='F'))
print('L = ',final_val(fit_arg_HCL.bval[1],fit_arg_HCL.sigma_bval[1],decimals=3,exp=-3,udm='Hr'))

---------------------------MODULO R -------------------------
chi2:  nan
---------------------------FASE R -------------------------


/tmp/ipykernel_7136/1741383604.py:2: RuntimeWarning: invalid value encountered in sqrt
  numerator = np.sqrt((1 - omega**2 * L * C) + omega**2 * C**2 * RL**2)
/usr/lib/python3/dist-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
